In [4]:
import math
import glob
import util as utils
import pose
import os
from pose_parser import parse_file, detect_perspective


from pprint import pprint
from scipy.signal import medfilt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import classification_report

In [3]:
files = utils.files_in_order('dataset/bicep')
X_train_names, X_test_names = train_test_split(files, test_size=0.4, random_state=42)
y_train = utils.get_labels(X_train_names)
y_test = utils.get_labels(X_test_names)
pprint(X_train_names)
pprint(y_train)
pprint(X_test_names)
pprint(y_test)

array(['bicep_bad_10.npy', 'bicep_bad_3.npy', 'bicep_good_6.npy',
       'bicep_bad_5.npy', 'bicep_bad_8.npy', 'bicep_good_1.npy',
       'bicep_good_3.npy', 'bicep_bad_4.npy', 'bicep_bad_7.npy'],
      dtype='<U16')
array([0, 0, 1, 0, 0, 1, 1, 0, 0])
array(['bicep_bad_1.npy', 'bicep_bad_2.npy', 'bicep_bad_6.npy',
       'bicep_good_5.npy', 'bicep_good_4.npy', 'bicep_good_2.npy',
       'bicep_bad_9.npy'], dtype='<U16')
array([0, 0, 0, 1, 1, 1, 0])


In [12]:
def load_features(names):
    upperarm_forearm_angles_list = []
    upperarm_torso_angles_list = []
    for filename in names:
        upperarm_forearm_angles = []
        upperarm_torso_angles = []
        frames = parse_file('dataset/bicep/' + filename)
        side = detect_perspective(frames)
        for frame in frames:
            if (side == pose.Side.right):
                upperarm = pose.Part(frame.relbow, frame.rshoulder)
                forearm = pose.Part(frame.relbow, frame.rwrist)
                torso = pose.Part(frame.rhip, frame.neck)
            else:
                upperarm = pose.Part(frame.lelbow, frame.lshoulder)
                forearm = pose.Part(frame.lelbow, frame.lwrist)
                torso = pose.Part(frame.lhip, frame.neck)
            angle1 = upperarm.calculate_angle(forearm)
            angle2 = upperarm.calculate_angle(torso)
            upperarm_forearm_angles.append(angle1)
            upperarm_torso_angles.append(angle2)
            upperarm_torso_angles_filtered = medfilt(medfilt(upperarm_torso_angles,5),5)
            upperarm_forearm_angles_filtered = medfilt(medfilt(upperarm_forearm_angles,5),5)
        upperarm_forearm_angles_list.append(upperarm_forearm_angles_filtered.tolist())
        upperarm_torso_angles_list.append(upperarm_torso_angles_filtered.tolist())

    return upperarm_forearm_angles_list,upperarm_torso_angles_list 


X_train_1, X_train_2 = load_features(X_train_names)
X_test_1, X_test_2 = load_features(X_test_names)

Data shape:  (80, 18, 3)
Mean torso:  382.9545547331215
Primary arm: Right


NameError: name 'upperarm_torso_angles' is not defined

In [11]:
p

[0.0]
